# Final Wave - Chahal.ipynb
### last updated: 12/14/23

#photo

In [ ]:
import numpy as np
import pandas as pd
from skimage import io
import cv2
from google.colab.patches import cv2_imshow # for image display without crashing colab
import matplotlib.pylab as plt

### Detecting edges

In [ ]:

#---------------USE------------- WAVE FINAL


img = io.imread('ocean-wave-0.jpeg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray,cmap = 'gray')

In [ ]:
from google.colab.patches import cv2_imshow # for image display without crashing colab
import matplotlib.pylab as plt

In [ ]:

#---------------USE-------------WAVE FINAL
#edges = cv2.Canny(img_gray,100,500) #input image, minVal, maxVal
                                    #final: parameters to mess with
edges = cv2.Canny(img_gray,100,415)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(img_gray,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122)
plt.imshow(edges,cmap = 'gray') # 'gray'
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()



In [ ]:
# print img pixel size:   use for making MASK
edges.shape

###STEP 2: Overlay 'Edge Image' on 'Original Image':




In [ ]:
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('ocean-wave-0.jpeg',0)
edges = cv2.Canny(img,50,200)   # canny edge detector

img = cv2.merge((img,img,img))  # creat RGB image from grayscale
img2 = img.copy()
img2[edges == 255] = [255, 0, 0]  # turn edges to red

plt.subplot(121),plt.imshow(img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img2)
plt.title('Edge Highlighted'), plt.xticks([]), plt.yticks([])

plt.show()

#video

In [ ]:
import cv2
import time
import os
import matplotlib.pyplot as plt

##step 1:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1) OPENCV2 SEPERATE INPUT VIDEO FRAME BY FRAME:

def ExtractFrames(path):
    #Creates a directory for frames
    folder = 'og_frames'
    os.makedirs(folder, exist_ok=True)

    # Path to video file
    vidObj = cv2.VideoCapture(path)

    # Used as counter variable for naming the image
    count = 0

    # checks whether frames were extracted
    success = 1

    # For setting sampling rate for the frame extraction
    #frame_rate = 30
    frame_rate = vidObj.get(cv2.CAP_PROP_FPS)
    print(frame_rate)

    prev = 0

    while success:
        # vidObj object calls read
        success, image = vidObj.read()

        # keep track of passed time
        time_elapsed = time.time() - prev

        # keep frames according to the set frame_rate
        if time_elapsed > 1./frame_rate:
          prev = time.time() # reset the timer function
          cv2.imwrite("og_frames/frame%d.jpg" % count, image) # save the file in a folder called frames with a unique name.
        # Saves the frames with frame-count
        count += 1


# Provide the path to your video file
path = "/content/kipahulu_original.mp4"

# Call the ExtractFrames function
ExtractFrames(path)


In [ ]:
# Rename filenames

import os

def rename_files_with_zero_padding(folder_path):
    # Get the list of files in the folder
    files = os.listdir(folder_path)

    # Rename each file with zero-padded numbers
    for old_name in files:
        # Extract the file extension
        base_name, extension = os.path.splitext(old_name)

        # Extract the numeric part of the filename
        numeric_part = base_name.replace('frame', '')

        # Convert the numeric part to an integer and format with zero-padding
        new_numeric_part = f"{int(numeric_part):03d}"

        # Construct the new name with zero-padded numbers
        new_name = f"frame{new_numeric_part}{extension}"

        # Construct the old and new paths
        old_path = os.path.join(folder_path, old_name)
        new_path = os.path.join(folder_path, new_name)

        # Rename the file
        os.rename(old_path, new_path)
        print(f"Renamed: {old_name} -> {new_name}")


folder_path = '/content/og_frames'
rename_files_with_zero_padding(folder_path)

##step 2:

In [ ]:
folder = '/content/og_frames'

for filename in os.listdir(folder):
  print(filename)

In [ ]:
# 2) TRANSFORM FRAMES: CANNY EDGE OVERLAY ON EVERY FRAME
# this is for if video is SPLIT FRAME BY FRAME, individ pics display egde overlay


#assign path of file with images for use in iterative process
folder = "/content/og_frames/"

folder1 = 'transform_frames1'
os.makedirs(folder1, exist_ok=True)
#count =0


#iterates over frames in the folder
for filename in os.listdir(folder):

  if filename.endswith(".jpg"):
    # Read the frame in grayscale
    frame_path = os.path.join(folder, filename)
    print(frame_path)
    img = cv2.imread(frame_path, 0)

    edges = cv2.Canny(img,50,200)   # canny edge detector

    img = cv2.merge((img,img,img))  # creat RGB image from grayscale
    img2 = img.copy()

    #  turn edges to red: *****
    img2[edges == 255] = [255, 0, 0]

# save img2 to a new folder directory "transformed image"
    # FURKAN OG  ****
    cv2.imwrite(f"/content/transform_frames1/{filename}", img2)

    # EMLYN *****
    #cv2.imwrite("/content/drive/MyDrive/resultgen/transform_frames/frame%d.jpg"%count, img2)
    #cv2.imwrite(f"/content/drive/MyDrive/resultgen/trans_frame/{filename}", img2)

    #print(count)
  #count += 1




##step 3:

In [ ]:
# 3) COMPILE BACK TO VIDEO
import cv2
import os

# set the path to the folder containing the frames
pathIn = '/content/transform_frames1/'

# ___CHANGE THIS EVERYTIME ---- set the output video file name and path

pathOut = '/content/kipahulu-wave-video-success.mp4'

# set the frame rate of the output video
fps = 30.0


# get the list of frame files
frameFiles = [f for f in os.listdir(pathIn) if f.endswith('.jpg')]


# sort the frame files in ascending order
frameFiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))


# get the size of the first frame
frameSize = cv2.imread(os.path.join(pathIn, frameFiles[0])).shape[1::-1]

# initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

videoWriter = cv2.VideoWriter(pathOut, fourcc, fps, frameSize)


for frameFile in frameFiles:
    # read the frame
    frame = cv2.imread(os.path.join(pathIn, frameFile))
    print(frameFile)
    # write the frame to the video
    videoWriter.write(frame)


# release the video writer
videoWriter.release()

In [ ]:
print(frameFiles)

In [ ]:
print(frameSize)